In [1]:
# imports
import torch, torchvision
import numpy as np
import utils
import my_models
import hyperparameters as h # this prints GPU enabled = True


# load the inputs as a list of ints
inputs, char2int_cypher = utils.load_music('input.txt', use_custom=True) 
# full input.txt is 501470 in length
dict_size = len(char2int_cypher) # conversion is the dict convert char to int

# define test and validation set
split = int(len(inputs) * 0.1) # change 0.1 to how big we want validation set to be
validation_set = inputs[:split]
training_set = inputs[split:]


# create model
lstm = my_models.lstm_char_rnn(dict_size, h.hidden_size, h.num_hidden_layers, h.batch_size)
init_hidden = lstm.initialize_hidden()
if h.GPU:
    init_hidden = init_hidden.cuda()
    lstm.cuda()
    
optimizer_lstm = torch.optim.Adam(lstm.parameters(), lr=0.01)

GPU enabled =  True
$
X:1
T: La Montfarine
Z:Transcrit et/ou corrig? par Michel BELLON - 2005-07-24
Z:Pour toute observation mailto:galouvielle@free.fr
M: 4/4
L: 1/8
Q:1/4=186
FGF B=AG G=AG F2F FGF {F}F2E EFE|
{E}E2D FGF B=AG G=AG {F}F2F FED C2G D2E|F3 {F}F/2 ED E3/2D/2|
EC FE E2 DC|DC C2 GD2E|F3F/2F/2 DE FD|EC B,C D2 B,G|
cB =A2 FG2E|F3F GF B=A|GG =AG F2 FF|GF F/2F3/2 EE FE|
E/2E3/2 DF GF B=A|GG =AG F/2F3/2 FF|ED C2 GD2E|F3F/2F/2 ED E3/2D/2|
EC FE E2 DC|DC C2 GD2E|F3F/2F/2 DE FD|EC B,C D2 B,G|
cB =A2 FF G=A|B3
G2_A G/2G3/2-|GF/2F/2 ED F2 EB,|CD EF GB AG|
A3A GA F2|F/2F/2F EF D3/2B,/2 B,B,|CD EF GA GF|G3B/2B/2 AB G2|
GG/2G/2 FG E2 EB/2B/2|AG BA GG AB|c3c/2c3/2d f2|ed eG c2 BB|
AB dc BG AD|E3F GF BA|GG AG F2 FF|GF F/2F3/2 EE FE|
E/2E3/2 DF GF BA|GG AG F/2F3/2 FF|ED C2 GD2E|F2- F/2
;
$
X: 2
T:Stella splendens
C


10


In [2]:
my_models.train(lstm, optimizer_lstm, h.epochs, inputs, h.sequence_length, init_hidden)


training epoch  0
seq  0
finished with this epoch!
training epoch  1
seq  0
finished with this epoch!
training epoch  2
seq  0
finished with this epoch!
training epoch  3
seq  0
finished with this epoch!
training epoch  4
seq  0
finished with this epoch!
training epoch  5
seq  0
finished with this epoch!
training epoch  6
seq  0
finished with this epoch!
training epoch  7
seq  0
finished with this epoch!
training epoch  8
seq  0
finished with this epoch!
training epoch  9
seq  0
finished with this epoch!


1